# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""12966594121660586144""",8.1394e18,"""../../../../data/SampleHDS.jso…","""16213733806433532114""","""12389604203113505438""","""7189849604827994599""",false
"""17909105717745695898""",3.2558e17,"""../../../../data/SampleHDS.jso…","""7368088096525236362""","""12389604203113505438""","""10411723875908756696""",true
"""2224057289082822490""",1.4181e19,"""../../../../data/SampleHDS.jso…","""5336126776549509997""","""4185235921171887873""","""7189849604827994599""",false
"""1197588711120825259""",1.7915e19,"""../../../../data/SampleHDS.jso…","""7368088096525236362""","""15348917492779909101""","""7189849604827994599""",false
"""14652648524834149339""",1.2550e19,"""../../../../data/SampleHDS.jso…","""16213733806433532114""","""15348917492779909101""","""7887436098792540635""",true
"""10180930787455143344""",1.9392e18,"""../../../../data/SampleHDS.jso…","""5336126776549509997""","""12695467595088885982""",null,false
"""11614391923100326336""",1.5737e19,"""../../../../data/SampleHDS.jso…","""5336126776549509997""","""15348917492779909101""","""7887436098792540635""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""2693063016261329210""",8.5424e18,"""../../../../data/SampleHDS.jso…","""2176186410263664439""","""FirstMortgage30yr""","""2480916800669711861""","""Rejected"""
"""13663209729075354870""",1.1573e19,"""../../../../data/SampleHDS.jso…","""6206444605637468390""","""FirstMortgage30yr""","""6148724870095980507""","""Accepted"""
"""13752064661150663353""",1.7837e19,"""../../../../data/SampleHDS.jso…","""13742469919383404989""","""MoneyMarketSavingsAccount""","""2480916800669711861""","""Rejected"""
"""4278909319353159294""",2.1048e18,"""../../../../data/SampleHDS.jso…","""6206444605637468390""","""BasicChecking""","""2480916800669711861""","""Rejected"""
"""5289473295907448034""",7.0451e18,"""../../../../data/SampleHDS.jso…","""2176186410263664439""","""BasicChecking""","""7992382043670570244""","""Accepted"""
"""14984668176722395866""",1.0940e19,"""../../../../data/SampleHDS.jso…","""13742469919383404989""","""UPlusFinPersonal""",null,"""Rejected"""
"""8020568563095088654""",1.6966e19,"""../../../../data/SampleHDS.jso…","""13742469919383404989""","""BasicChecking""","""7992382043670570244""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""15817344893893706607""",6.4529e18,"""../../../../data/SampleHDS.jso…","""5428569341997076199""","""FirstMortgage30yr""","""8077201405311366912""","""Rejected"""
"""9455517170590553341""",7.2162e18,"""../../../../data/SampleHDS.jso…","""16361791012203088208""","""FirstMortgage30yr""","""3563248967714237740""","""Accepted"""
"""2757873603826598519""",1.7725e19,"""../../../../data/SampleHDS.jso…","""12998887436935034178""","""MoneyMarketSavingsAccount""","""8077201405311366912""","""Rejected"""
"""2769473506539838498""",4.8717e18,"""../../../../data/SampleHDS.jso…","""16361791012203088208""","""BasicChecking""","""8077201405311366912""","""Rejected"""
"""14307498553172057419""",5.5270e18,"""../../../../data/SampleHDS.jso…","""5428569341997076199""","""BasicChecking""","""14638340152789677911""","""Accepted"""
"""12104382896446018419""",1.7970e19,"""../../../../data/SampleHDS.jso…","""12998887436935034178""","""UPlusFinPersonal""",null,"""Rejected"""
"""7909492205048073676""",1.8294e19,"""../../../../data/SampleHDS.jso…","""12998887436935034178""","""BasicChecking""","""14638340152789677911""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""11607069941383855728""",6.0221e18,"""6488086644001720326""","""FirstMortgage30yr""","""3813394123237880986""","""Rejected"""
"""18278964075147118547""",2.2228e18,"""6077337721412885530""","""FirstMortgage30yr""","""3359379319836418694""","""Accepted"""
"""14791683637853151924""",6.8988e18,"""4239025813693951965""","""MoneyMarketSavingsAccount""","""3813394123237880986""","""Rejected"""
"""16287708547505018726""",3.2804e18,"""6077337721412885530""","""BasicChecking""","""3813394123237880986""","""Rejected"""
"""8177611666367621678""",5.4996e18,"""6488086644001720326""","""BasicChecking""","""18378854800664855497""","""Accepted"""
"""15369738107837082215""",9.6513e18,"""4239025813693951965""","""UPlusFinPersonal""",null,"""Rejected"""
"""13418639920439002120""",1.3739e19,"""4239025813693951965""","""BasicChecking""","""18378854800664855497""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'